In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from keras import models
from keras import layers
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
X = train.drop('label', axis=1)
y = train['label']

In [4]:
X = X/255

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))

Input shape: [784]


In [5]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
x_test = x_test.values.reshape(-1, 28, 28, 1)

In [6]:
model = keras.Sequential([

    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same', input_shape=(28, 28, 1)),
    layers.MaxPool2D(pool_size=2),
    
    layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPool2D(pool_size=2),
    
    layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(32,activation = 'relu'),
    layers.Dense(16,activation='relu'),
    layers.Dense(10,activation = 'softmax')
    
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 28, 28, 128)       1280      
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 14, 14, 128)       0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 12, 12, 64)        73792     
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, 6, 6, 64)          0         
_________________________________________________________________
module_wrapper_4 (ModuleWrap (None, 4, 4, 64)          36928     
_________________________________________________________________
module_wrapper_5 (ModuleWrap (None, 1024)              0         
_________________________________________________________________
module_wrapper_6 (ModuleWrap (None, 64)                6

In [10]:
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [11]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,
    patience=20,
)

history = model.fit(
    x_train, y_train ,
    epochs=20,
    callbacks=[early_stopping],
    batch_size=200,#64
    validation_data =(x_test,y_test)
    
)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/module_wrapper/conv2d/Conv2D (defined at C:\Users\Lee\anaconda3\lib\site-packages\keras\layers\convolutional.py:245) ]] [Op:__inference_train_function_1736]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/module_wrapper/conv2d/Conv2D:
 IteratorGetNext (defined at <ipython-input-11-8ce08e31871f>:6)

Function call stack:
train_function


In [ ]:
test_reshape = test.values.reshape(-1, 28, 28, 1)

In [ ]:
test_reshape

In [ ]:
submit = pd.DataFrame(np.argmax(model.predict(test_reshape), axis=1), columns=['Label'], 
                      index=pd.read_csv('./sample_submission.csv')['ImageId'])

In [ ]:
submit.index.name = 'ImageID'
submit.to_csv('submission2.csv')

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()